# Predicting the Holdout Set

In [1]:
#importing libraries
import pandas as pd
import numpy as np 
import pickle

pd.set_option('display.max_columns', 300)

In [2]:
#reading in DataFrame
df = pd.read_csv('Data/kc_house_data_test_features.csv', index_col = 'Unnamed: 0')

df.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,1974300020,20140827T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
1,1974300020,20150218T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
2,3630020380,20141107T000000,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,98029,47.5472,-121.998,1470,1576
3,1771000290,20141203T000000,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,98077,47.7427,-122.071,1160,10565
4,5126310470,20150115T000000,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,98059,47.4863,-122.140,2830,7916


### Bringing in the Model and Saved Variables

In [3]:
#bringing in the model
infile = open("model.pickle",'rb')
model = pickle.load(infile)
infile.close()

In [4]:
#bringing in the saved variables
infile = open("other_info.pickle",'rb')
other_info = pickle.load(infile)
infile.close()

In [5]:
other_info[1] #I'm doing this to condense the notebook space

2.0

In [6]:
#assigning the other_info variables to new names:
bedroom_mean, bathroom_median, sqft_living_median, sqft_above_median, yard_size_mean, column_list = other_info

### Cleaning

In [7]:
df.isna().sum().head(2)

id      0
date    0
dtype: int64

In [8]:
df.dtypes.head()

id               int64
date            object
bedrooms         int64
bathrooms      float64
sqft_living      int64
dtype: object

In [9]:
#CLEANING THE COLUMNS

#DATE
df['date'] = pd.to_datetime(df['date'])

#BEDROOM
conditions = [df['bedrooms'] == 33, df['bedrooms'] > 8, df['bedrooms'] < 1]
choices = [bedroom_mean, 8, bedroom_mean]
df['bedrooms'] = np.select(conditions, choices, df['bedrooms'])

#BATHROOM
df.bathrooms = round(df.bathrooms, 0)
df.bathrooms=np.where(df['bathrooms']==0, bathroom_median, df['bathrooms'])

#SQFT_LIVING
df.sqft_living = np.where(df['sqft_living'] == 13540,sqft_living_median, df['sqft_living'])

#SQFT_ABOVE
df.sqft_above = np.where(df['sqft_above'] == 9410, sqft_above_median, df['sqft_above'])



#BRINGING IN NEW FEATURES:
#MONTH
df['month'] = df['date'].dt.month

#BEEN RENOVATED
df['been_renovated']=np.where(df['yr_renovated'] == 0, 0, 1)

#YARD SIZE
df['yard_size'] = df['sqft_lot'] - (df['sqft_living']/df['floors'])
df['yard_size']=np.where(df['yard_size'] < 0, yard_size_mean, df['yard_size'])

#LOG YARD
df['log_yard'] = np.log(df['yard_size'])

#NEIGHBOR
df['neighbor'] = df['sqft_living15']*df['sqft_lot15']

#LOG NEIGHBOR
df['log_neighbor'] = np.log(df['neighbor'])

#RENO BATH
df['reno_bath'] = df['been_renovated']*df['bathrooms']

#bed*bath
df['bedbath'] = df['bedrooms']*df['bathrooms']

#sqft_living**2
df['sqft_living2'] = df['sqft_living'] * df['sqft_living']

#sqft_living15**2
df['sqft_living152'] = df['sqft_living15'] * df['sqft_living15']

#floor**2
df['floors2'] = df['floors']*df['floors']


### Dropping Columns & Getting Dummies

In [10]:
# DROPPING UNUSED COLUMNS

#unused columns:
df.drop(columns=['id', 'date', 'lat', 'long', 'yr_built', 'sqft_lot', 'condition'], inplace = True)

#issues with multicollinearity:
#the threshold is at .9
df.drop(columns=['yr_renovated', 'been_renovated', 'sqft_lot15', 'yard_size', 'neighbor',
                 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_living15', 'floors'], inplace = True)


# GETTING DUMMIES
#creating dummies for the categorical values
final_df = pd.get_dummies(df, columns = ['view','grade', 'month', 'zipcode'], drop_first = True)


### Checking If the Columns Match:

In [11]:
#columns in the holdout set
holdout = list(final_df.columns)

In [12]:
#these are the columns in the train NOT in the holdout. 
need_cols = []
for i in column_list:      #these are the columns in the model df from the EDA notebook
    if i not in holdout:
        need_cols.append(i)
        
#These columns need to be added
print(need_cols)

['grade_3']


In [13]:
def add_fake_columns(need_cols):
    """
    Creates fake columns for the ones in the holdout set that are not
    in the test set
    """
    for col in need_cols:
        final_df[col]=np.zeros(final_df.shape[0]).astype('uint8')
    return final_df

In [14]:
final_df = add_fake_columns(need_cols)

# Predicting the Holdout

In [15]:
#getting predictions
final_answers = model.predict(final_df)

#saving predictions to new DataFrame
df = pd.DataFrame(final_answers)
df.to_csv('Data/housing_preds_zachary_greenberg.csv')